# Test of Tetris Reinforcement Learning

In [1]:
# Import necessary libraries
import torch
import cv2
from src.tetris import Tetris

In [2]:
# helper class that serves as a container for various configuration options for tetris and agent
class Options:
    def __init__(self, width, height, block_size, fps, saved_path, output):
        self.width = width
        self.height = height
        self.block_size = block_size
        self.fps = fps
        self.saved_path = saved_path
        self.output = output

In [3]:
# test the AI model in the game of Tetris
def test(opt):
    # seed for random number generation to ensure reproducibility of results
    if torch.cuda.is_available():
        torch.cuda.manual_seed(256)
    else:
        torch.manual_seed(256)
        
    # load trained model. If CUDA is available model is loaded into GPU. Otherwise it is loaded into CPU   
    if torch.cuda.is_available():
        model = torch.load("{}/tetris".format(opt.saved_path))
    else:
        model = torch.load("{}/tetris".format(opt.saved_path), map_location=lambda storage, loc: storage)
    model.eval()
    
    # create new game of tetris with the specified parameters    
    env = Tetris(width=opt.width, height=opt.height, block_size=opt.block_size)
    env.reset()
    
    # move model to GPU if available
    if torch.cuda.is_available():
        model.cuda()
        
    # create video writer object. Will output a video file of agent playing the game 
    out = cv2.VideoWriter(opt.output, cv2.VideoWriter_fourcc(*"MJPG"), opt.fps,
                          (int(1.5*opt.width*opt.block_size), opt.height*opt.block_size))
    while True:
        # get next possible states of the game    
        next_steps = env.get_next_states()
        next_actions, next_states = zip(*next_steps.items())
        next_states = torch.stack(next_states)
        
        # move the states to GPU if available
        if torch.cuda.is_available():
            next_states = next_states.cuda()
            
        # model makes predictions based on next possible states
        predictions = model(next_states)[:, 0]
        
        # action with highest prediction value is chosen
        index = torch.argmax(predictions).item()
        action = next_actions[index]
        
        # perform the chosen action in the game
        _, done = env.step(action, render=True, video=out)

        # if game is over close video writer and exit loop
        if done:
            out.release()
            break

In [4]:
# manually setting the parameters
opt = Options(width=10, height=20, block_size=30, fps=300, saved_path="trained_models", output="output.mp4")
test(opt)

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
